In [7]:
from bs4 import BeautifulSoup
import requests
import random
from time import sleep

%env https_proxy=http://172.16.2.250:3128

env: https_proxy=http://172.16.2.250:3128


In [24]:
def scrape_from_page(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    if(soup.find("h1", class_ = "a_t") is None): return []

    title = soup.find("h1", class_ = "a_t").get_text()
    date = soup.find("a", id = "article_date_p").attrs['data-date'][:10]

    summary = soup.find("h2", class_ = "a_st").get_text()
    content = "".join([elt.text for elt in soup.find_all("p")])[:-62]

    return [title, date, url, summary, content]

In [13]:
def get_topic_pages(page):
  soup = BeautifulSoup(page.content, "html.parser")
  urls_1 = list(set([elt.find("a")["href"] for elt in soup.find_all("article", class_='c c-d c--m')]))
  urls_2 = list(set([elt.find("a")["href"] for elt in soup.find_all("article", class_='c c-o c-d c--m')]))
  urls_incomplete = urls_1 + urls_2
  urls = [('https://english.elpais.com' + urls_incomplete[i]) for i in range(len(urls_incomplete))]
  return urls

In [25]:
def scrape(page_no, topic):
  URL = f"https://english.elpais.com/{topic}/{str(page_no)}"
  url_list = get_topic_pages(requests.get(URL))
  sleep(1)

  data = []

  for url in url_list:
    if (scrape_from_page(url)):
      data.append(scrape_from_page(url))
      sleep(1)

  return data

In [26]:
def scrape_loop(start_pg, end_pg, topic):
    data = []
    for pg in range(start_pg, end_pg):
        data += scrape(pg, topic)
        
    return data

In [27]:
dt = scrape_loop(1, 2, "sports")

In [ ]:
  url = "https://english.elpais.com/{topic}/{str(page_no)}"
  page = requests.get(url)
  soup = BeautifulSoup(page.content, "html.parser")